In [86]:
## Austin Caudill
## 03-07-2023

## Install required Packages:
#!pip install simplekml

In [87]:
import os
import pandas as pd
import simplekml

from simplekml import Region, LatLonAltBox, Lod ## Not sure if all this is needed.

os.chdir("C:/Users/Austin/Daylight Petroleum Dropbox/Austin Caudill/Well Master")

# Load well data from CSV file
well_data = pd.read_excel('DLP Well Master.xlsx')

os.chdir("C:/Users/Austin/Documents/GitHub/Oil-and-Gas-KML-Generator")

# Create a KML object
kml = simplekml.Kml()

In [88]:
# Define icon URLs for different well types and statuses with Python Dictionaries.

icon_urls = {
    ('OIL WELL', 'ACTIVE'): 'Icons/Active/OIL WELL.png',
    ('OIL WELL', 'INACTIVE'): 'Icons/Inactive/OIL WELL.png',
    ('OIL WELL', 'TA'): 'Icons/Inactive/OIL WELL.png',
    ('OIL WELL', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('GAS WELL', 'ACTIVE'): 'Icons/Active/GAS WELL.png',
    ('GAS WELL', 'INACTIVE'): 'Icons/Inactive/GAS WELL.png',
    ('GAS WELL', 'TA'): 'Icons/Inactive/GAS WELL.png',
    ('GAS WELL', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('INJECTION WELL', 'ACTIVE'): 'Icons/Active/WATER.png',
    ('INJECTION WELL', 'INACTIVE'): 'Icons/Inactive/WATER.png',
    ('INJECTION WELL', 'TA'): 'Icons/Inactive/WATER.png',
    ('INJECTION WELL', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('SWD', 'ACTIVE'): 'Icons/Active/WATER.png',
    ('SWD', 'TA'): 'Icons/Inactive/WATER.png',    
    ('SWD', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('WSW', 'ACTIVE'): 'Icons/Active/WATER.png',
    ('WSW', 'INACTIVE'): 'Icons/Inactive/WATER.png',
    ('WSW', 'TA'): 'Icons/Inactive/WATER.png',
}


In [89]:
import simplekml
import pandas as pd

# Create a folder for divested wells
divested_folder = kml.newfolder(name="Divested Wells")

# Dictionary to store folders per District to avoid creating duplicates
divested_district_folders = {}

# Loop through each unique combination of District, Area, Route, and EngGroup
for district, district_group in well_data.groupby('District'):
    # Create the regular folder for the district
    district_folder = kml.newfolder(name=district)

    # Track if we need to create a divested district folder
    has_divested_wells_in_district = False

    for area, area_group in district_group.groupby('Area'):
        area_folder = district_folder.newfolder(name=area)

        # Track if we need to create a divested area folder
        has_divested_wells_in_area = False

        for route, route_group in area_group.groupby('Route'):
            route_folder = area_folder.newfolder(name=route)

            # Track if we need to create a divested route folder
            has_divested_wells_in_route = False

            for enggroup, group in route_group.groupby('EngGroup'):
                enggroup_folder = route_folder.newfolder(name=enggroup)

                # Check if there are divested wells in this EngGroup
                divested_wells_in_group = group['DIV_CLOSE_DATE'].notna().any()

                if divested_wells_in_group:
                    # Create the divested folder structure only if we have divested wells
                    if not has_divested_wells_in_district:
                        divested_district_folders[district] = divested_folder.newfolder(name=district)
                        has_divested_wells_in_district = True

                    if not has_divested_wells_in_area:
                        divested_area_folder = divested_district_folders[district].newfolder(name=area)
                        has_divested_wells_in_area = True

                    if not has_divested_wells_in_route:
                        divested_route_folder = divested_area_folder.newfolder(name=route)
                        has_divested_wells_in_route = True

                    # Create or reuse the divested enggroup folder
                    divested_enggroup_folder = divested_route_folder.newfolder(name=enggroup)

                # Loop through each well in this group
                for index, row in group.iterrows():
                    is_divested = pd.notnull(row['DIV_CLOSE_DATE'])

                    # Set the appropriate folder for the well
                    well_folder = divested_enggroup_folder if is_divested else enggroup_folder

                    # Get well details
                    well_name = row['WellName']
                    lat = row['Lat']
                    lon = row['Long']
                    coords = [(lon, lat)]

                    # Create the point in the appropriate folder
                    point = well_folder.newpoint(name=well_name, coords=coords)

                    # Set the icon image for the point based on WellType and Well_Status values
                    icon_url = icon_urls.get((row['WellType'], row['WELL_STATUS']), "Icons/Unknown.png")
                    point.style.iconstyle.icon = simplekml.Icon(href=icon_url)

                    # Add description
                    description = f'<b>API:</b> {row["API"]}<br>' \
                                  f'<b>District:</b> {row["District"]}<br>' \
                                  f'<b>Area:</b> {row["Area"]}<br>' \
                                  f'<b>Route:</b> {row["Route"]}<br>' \
                                  f'<b>Eng Group:</b> {row["EngGroup"]}<br>' \
                                  f'<b>County:</b> {row["County"]}<br>' \
                                  f'<b>State:</b> {row["STATE"]}<br>' \
                                  f'<b>AL Type:</b> {row["AL_Type"]}<br>' \
                                  f'<b>Production Engineer:</b> {row["ProdEngineer"]}<br>'

                    # Add divestiture information if applicable
                    if is_divested:
                        description += f'<b>New Operator:</b> {row["Operator"]}<br>' \
                                       f'<b>Divestiture Effective Date:</b> {row["DIV_EFF_DATE"]}<br>' \
                                       f'<b>Divestiture Close Date:</b> {row["DIV_CLOSE_DATE"]}<br>' \
                                       f'<b>Divestiture Group:</b> {row["DivestitureGroup"]}<br>'

                    point.description = description


In [90]:
# Save the KML file
kml.save('well_data.kml')

In [91]:
# Define the path to the KML file
kml_path = 'well_data.kml'

# Save the KML file
kml.save(kml_path)

print("Finished.")

Finished.
